In [1]:
"""
This is the main code for the General Adversarial Network
Trains GAN as specified in the /architecture folder
Searches for best fit hyper-parameters 
"""

'\nThis is the main code for the General Adversarial Network\nTrains GAN as specified in the /architecture folder\nSearches for best fit hyper-parameters \n'

In [2]:
# =========
# TODO:
# network visualiztion https://lutzroeder.github.io/netron/
# any object dataset
# assert shape
# shift tab ??

In [3]:
"""
Library: general
"""
import time
import datetime
import math
import os
import logging

In [4]:
"""
Library: randomness
"""
import random
import scipy 
import numpy as np
import tensorflow as tf
import keras 

tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [5]:
"""
Result repdoducibility
"""
RANDOM_SEED = 0
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.set_random_seed(RANDOM_SEED)

In [6]:
"""
Versioning
"""
print("SCIPY=", scipy.__version__)
print("NUMPY=", np.__version__)
print("TENSORFLOW=", tf.__version__)
print("KERAS=", keras.__version__)

SCIPY= 1.3.1
NUMPY= 1.16.4
TENSORFLOW= 1.14.0
KERAS= 2.2.4


In [7]:
"""
www.wandb.com experiment tracking installation 
"""

# !pip install graphql-core==2.0
# !pip install wandb

'\nwww.wandb.com experiment tracking installation \n'

In [8]:
"""
www.wandb.com experiment tracking loging 
"""

import wandb

!wandb login 45396bf25753eeeb051e5567c5e7dd67446e3be4

wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
Successfully logged in to Weights & Biases!


In [9]:
"""
Plotting dependencies
"""

# !pip install pydot

'\nPlotting dependencies\n'

In [10]:
"""
Load architecture for Discriminator and Generator
"""
from architecture import atienza_G as atienza_G
from architecture import atienza_D as atienza_D
from architecture import atienza_bertorello_G as atiber_G
from architecture import atienza_bertorello_D as atiber_D
from architecture import brownlee_G as brownlee_G
from architecture import brownlee_D as brownlee_D


def get_G(architecture_choice, input_n):
    if architecture_choice == 'atienza':
        return atienza_G.DifferetiableNetwork(input_n)
    if architecture_choice == 'atiber':
        return atiber_G.DifferetiableNetwork(input_n)
    if architecture_choice == 'brownlee':
        return brownlee_G.DifferetiableNetwork(input_n)
        
def get_D(architecture_choice, object_shape):    
    if architecture_choice == 'atienza':
        return atienza_D.DifferetiableNetwork(object_shape)
    if architecture_choice == 'atiber':
        return atiber_D.DifferetiableNetwork(object_shape)
    if architecture_choice == 'brownlee':
        return brownlee_D.DifferetiableNetwork(object_shape)


In [11]:
"""
Loss functions
https://github.com/TwistedW/tf-GANs-Loss/blob/master/GAN_Loss.py
"""
import tensorflow as tf
import tensorflow.contrib as tf_contrib

def discriminator_loss(loss_func, real, fake):
    real_loss = 0
    fake_loss = 0

    if loss_func.__contains__('wgan'):
        real_loss = -tf.reduce_mean(real)
        fake_loss = tf.reduce_mean(fake)

    if loss_func == 'lsgan':
        real_loss = tf.reduce_mean(tf.squared_difference(real, 1.0))
        fake_loss = tf.reduce_mean(tf.square(fake))

    if loss_func == 'gan' or loss_func == 'dragan':
        real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(real), logits=real))
        fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(fake), logits=fake))

    if loss_func == 'hinge':
        real_loss = tf.reduce_mean(relu(1.0 - real))
        fake_loss = tf.reduce_mean(relu(1.0 + fake))

    loss = real_loss + fake_loss

    return loss


def generator_loss(loss_func, fake):
    fake_loss = 0

    if loss_func.__contains__('wgan'):
        fake_loss = -tf.reduce_mean(fake)

    if loss_func == 'lsgan':
        fake_loss = tf.reduce_mean(tf.squared_difference(fake, 1.0))

    if loss_func == 'gan' or loss_func == 'dragan':
        fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(fake), logits=fake))

    if loss_func == 'hinge':
        fake_loss = -tf.reduce_mean(fake)

    loss = fake_loss

    return loss

def gradient_penalty(self, real, fake):
    if self.gan_type == 'dragan':
        shape = tf.shape(real)
        eps = tf.random_uniform(shape=shape, minval=0., maxval=1.)
        x_mean, x_var = tf.nn.moments(real, axes=[0, 1, 2, 3])
        x_std = tf.sqrt(x_var)  # magnitude of noise decides the size of local region
        noise = 0.5 * x_std * eps  # delta in paper

        # Author suggested U[0,1] in original paper, but he admitted it is bug in github
        # (https://github.com/kodalinaveen3/DRAGAN). It should be two-sided.

        alpha = tf.random_uniform(shape=[shape[0], 1, 1, 1], minval=-1., maxval=1.)
        interpolated = tf.clip_by_value(real + alpha * noise, -1., 1.)  # x_hat should be in the space of X

    else:
        alpha = tf.random_uniform(shape=[self.batch_size, 1, 1, 1], minval=0., maxval=1.)
        interpolated = alpha * real + (1. - alpha) * fake

    logit = self.discriminator(interpolated, reuse=True)

    grad = tf.gradients(logit, interpolated)[0]  # gradient of D(interpolated)
    grad_norm = tf.norm(flatten(grad), axis=1)  # l2 norm

    GP = 0

    # WGAN - LP
    if self.gan_type == 'wgan-lp':
        GP = self.ld * tf.reduce_mean(tf.square(tf.maximum(0.0, grad_norm - 1.)))

    elif self.gan_type == 'wgan-gp' or self.gan_type == 'dragan':
        GP = self.ld * tf.reduce_mean(tf.square(grad_norm - 1.))

    return GP

In [12]:
"""
A note on training GANs
"""
# https://github.com/eriklindernoren/Keras-GAN
# https://www.youtube.com/watch?v=XeQBsidyhWE
# https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy
# https://machinelearningmastery.com/how-to-code-the-generative-adversarial-network-training-algorithm-and-loss-functions/
# https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618

# def CrossEntropyLoss(yHat, y):
#     if y == 1:
#       return -log(yHat)       // loss rapidly increasing with yHat -> 0
#     else:
#       return -log(1-yHat)     // loss rapidly increasing with yHat -> 1

# D is trained with binary_crossentropy:
# a) true x examples with label y=1, and thus yHat=D(x)
# b) fake G(z) examples with label y=0, and thus yHat=D(G(z))
# Thus D loss in heavyside form:
# Example loss = yTrue*log(D(x)) + yGen*log(1−D(G(z)))
# minimize J = - Sum(loss) / m

# GD is trained with binary_crossentropy (to fool D):
# a) with discriminator fake label y=1, and thus yHat=D(G(z))
# b) no training with label y=0
# Thus G loss is defined heavyside: 
# Example loss = 0 + yGen*log(1-D(G(z))) 
# minimize J = Sum(loss) / m

# For faster training of D, with stronger gradients: 
# Example loss = log(D(G(z)))
# J = - Sum(loss) / m 


'\nA note on training GANs\n'

In [13]:
"""
Keras custom loss definition example
"""
# # Define custom loss
# def custom_loss(layer):

#     # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
#     def strong_gradient_cross_entropy_loss(y_true, y_pred):
#         return K.mean(K.square(y_pred - y_true) + K.square(layer), axis=-1)
   
#     return strong_gradient_cross_entropy_loss

'\nKeras custom loss definition example\n'

In [14]:
"""
Assemble a GAN from: Discriminator, Generator and other parameters
"""
import onenow_gan_factory_adversarial as adversarial_factory
import onenow_gan_factory_sequential as sequential_factory

    
def get_adversarial_net(architecture_choice, optimizer_type, learning_rate, 
                        object_shape, G_input_n, net_config,
                        transfer_D_path, transfer_GD_path,
                        D_num_frozen_layers, GD_num_frozen_layers):
    """
    D is trained to: a) correctly label real example as 1, and b) correctly label fake example as 0
    Success for G succeeds fooling D, generating fake images indistiguishable from real ones
    Success for D is detecting fakes; ultimately accuracy converges to 50% (fakes as good as real) 
    """        
    # binary cross-entropy loss 
    discriminator_loss_function = 'binary_crossentropy' # custom_loss
    discriminator_learning_metrics = ['accuracy']

    # optimizer: Generator
    generator_loss_function = 'binary_crossentropy'
    generator_learning_metrics = ['accuracy'] 

    G_net = get_G(architecture_choice, G_input_n) 
    D_net = get_D(architecture_choice, object_shape)
                        
    # model
    # TODO: rename Sequential_Network to NetworkModel 
    discriminator_model = sequential_factory.Sequential_Network("D_", [D_net], 
                optimizer_type, learning_rate, 
                discriminator_loss_function, discriminator_learning_metrics,
                net_config['discriminator_model_path'], transfer_D_path, D_num_frozen_layers)
    
    # prevent discriminator from converging much faster than generator
    generator_learning_fraction = 2 # k TODO: turn into hyper param, use to inner loop instead
    generator_learning_rate = learning_rate / generator_learning_fraction
    
    generator_model = sequential_factory.Sequential_Network("GD_", [G_net, D_net], 
                optimizer_type, generator_learning_rate, 
                generator_loss_function, generator_learning_metrics,
                net_config['generator_model_path'], transfer_GD_path, GD_num_frozen_layers)

    return adversarial_factory.Adversarial_Network(generator_model, discriminator_model), \
         discriminator_model, generator_model   
        

In [15]:
"""
Dataset augmentation libarry
"""
# !pip install imgaug

'\nDataset augmentation libarry\n'

In [16]:
# https://github.com/aleju/imgaug
import imgaug as ia
import imgaug.augmenters as iaa

ia.seed(1)

# https://imgaug.readthedocs.io/en/latest/source/examples_basics.html
# It applies crops and affine transformations to images, flips some of the images horizontally, 
# adds a bit of noise and blur and also changes the contrast as well as brightness.

crop_and_afine_transformation = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order

In [17]:
"""
Load a dataset: MNIST, CIFAR, etc
"""
import onenow_gan_utils as outil 

import imgaug as ia
import imgaug.augmenters as iaa
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# https://www.tensorflow.org/api_docs/python/tf/keras/datasets
# https://github.com/astorfi/TensorFlow-World/tree/master/docs/tutorials/3-neural_network/autoencoder
def get_data_set(dataset_name, class_list, is_augment, downsample_percent):
    
    ## dataset
    if dataset_name == "mnist":
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    if dataset_name == "cifar10":
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()                

    ## class: filter 
    print("x_train_before_class_filter=", x_train.shape)        
    train_filter = []
    for i in range(len(x_train)):
        if y_train[i] in class_list:
            train_filter.append(i)
    x_train = x_train[train_filter]            
    y_train = y_train[train_filter] 
    
    ## downsample
    downsample_count = len(y_train) * downsample_percent 
    idx = np.random.choice(len(y_train), int(downsample_count))
    x_train = x_train[idx]
    print("x_train_after_downsample=", x_train.shape)        

    ## augment    
    if is_augment:
        x_all_augmented = x_train
        for i in range(10): # augmenters are applied in random order
            x_train_augmented = crop_and_afine_transformation(images=x_train)
            x_all_augmented = np.concatenate((x_all_augmented , x_train_augmented), axis=0)
        x_train = x_all_augmented    
    print("x_train_after_augmentation=", x_train.shape)

    ## process: normalize
    # convert from unsigned ints to floats
    x_train = x_train.astype('float32')
    # scale from [0,255] to [-1,1]
    x_train = (x_train - 127.5) / 127.5
    print("x_train_after_class_filter=", x_train.shape)

    return x_train, x_train[0].shape


choice_class = 7
choice_pic = 2

train_original, shape_original = get_data_set("cifar10", [choice_class], is_augment=False, downsample_percent=0.1)
outil.plot_one_object(train_original[choice_pic], "save_me_original", "./")

train_augmented, shape_augmented = get_data_set("cifar10", [choice_class], is_augment=True, downsample_percent=0.1)
outil.plot_one_object(train_augmented[len(train_original)+choice_pic], "save_me_augmented", "./")


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


x_train_before_class_filter= (50000, 32, 32, 3)
x_train_after_downsample= (500, 32, 32, 3)
x_train_after_augmentation= (500, 32, 32, 3)
x_train_after_class_filter= (500, 32, 32, 3)


<Figure size 640x480 with 0 Axes>

x_train_before_class_filter= (50000, 32, 32, 3)
x_train_after_downsample= (500, 32, 32, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


x_train_after_augmentation= (5500, 32, 32, 3)
x_train_after_class_filter= (5500, 32, 32, 3)


<Figure size 640x480 with 0 Axes>

In [18]:
"""
Run the GAN training
"""

from math import sqrt

from onenow_gan_trainer import GAN_TRAINER


def run_train(x_train, object_shape, architecture_choice, optimizer_type, learning_rate, net_config, logger,
              transfer_D_path, transfer_GD_path, D_num_frozen_layers, GD_num_frozen_layers, is_run_train = True):
    # config
    wandb.init(project=net_config['project_name'])
    wandb.config.update(net_config)
    # transfer
    G_input_n = 100
    A_net, discriminator_model, generator_model  = \
            get_adversarial_net(architecture_choice, optimizer_type, learning_rate, 
                                object_shape, G_input_n, net_config,
                                transfer_D_path, transfer_GD_path,
                                D_num_frozen_layers, GD_num_frozen_layers)
    # train
    trainer = GAN_TRAINER(x_train, A_net, net_config, logger) 
    if is_run_train:
        trainer.train(net_config['train_batch_size'], G_input_n)   
    return A_net    


In [19]:
"""
MAJORITY CLASS PART1: Train CIFAR10 GAN Generator for all majority classes
"""
from onenow_gan_config import SystemConfig


if __name__ == '__main__':
    """
    Search for best model by iterating over hyper-parameters
    """
    
    # init
    version = "23"
    name = "gan-cifar10-012345678-allfreeze"
    project_name = (name + "__v%s") % str(format(int(version), '03d'))
    global_config = SystemConfig(project_name)
    wandb_config = global_config.get_config()
    logger = logging.getLogger()    

    # define hyper param space
    hyperparam_space = {}  
    hyperparam_space['data_set'] = ['cifar10'] #  'mnist'    
    hyperparam_space['class_list'] = [0,1,2,3,4,5,6,7,8] 
    hyperparam_space['architecture_choice'] = ['brownlee'] #  'atienza', 'atiber'
    hyperparam_space['optimizer_type'] = ['Adam']  # ', 'RMSprop'
    hyperparam_space['learning_rate'] = [0.0002]  # 0.01*10**(-2), 0.1*10**(-2), 0.001*10**(-2)
    hyperparam_space['batch_size'] = [1024] # 128, 4096, 8192, 1024, 256, 
    
    
    # iterate over hyper param space    
    for dataset_name in hyperparam_space['data_set']:
        for arch_choice in hyperparam_space['architecture_choice']:
            wandb_config['architecture_choice'] = arch_choice
            for optimizer_type in hyperparam_space['optimizer_type']: 
                wandb_config['optimizer_type'] = optimizer_type
                for learning_rate in hyperparam_space['learning_rate']:
                    wandb_config['learning_rate'] = learning_rate
                    for train_batch_size in hyperparam_space['batch_size']:
                        wandb_config['train_batch_size'] = train_batch_size
                        ## path   
                        project_tag = '__data_set=' + dataset_name
                        class_string = ""
                        for k in sorted(hyperparam_space['class_list']):
                            class_string += str(k) 
                        project_tag += '__class_list=' + class_string
                        project_tag += '__architecture=' + arch_choice 
                        project_tag += '__optimizer=' + optimizer_type 
                        project_tag += '__learningrate='+ str(learning_rate) 
                        project_tag += '__batchsize=' + str(train_batch_size) 
                        wandb_config['project_tag'] = project_tag
                        print("project_name=" + project_name + "\t"+ "project_tag=" + project_tag)
                        ## file
                        global_config.set_log_file(logger, project_tag)
                        global_config.set_creation_folder(project_tag)
                        global_config.set_model_folder(project_tag)
                        ## data set
                        x_train, object_shape = get_data_set(dataset_name, hyperparam_space['class_list'],
                                                             is_augment = False, downsample_percent=0.1) 
                        ## transfer learning
                        # transfer_D_path = ""
                        # transfer_GD_path = ""
                        transfer_D_path = "/home/ec2-user/SageMaker/Rosenblatt-AI/CS230/latest/discriminator.h5.latest.0-8"
                        transfer_GD_path = "/home/ec2-user/SageMaker/Rosenblatt-AI/CS230/latest/generator.h5.latest.0-8"
                        # transfer_D_path = "/home/ec2-user/SageMaker/efs/model/gan-cifar10-012345678__v019__data_set=cifar10__class_list=012345678__architecture=brownlee__optimizer=RMSprop__learningrate=0.0001__batchsize=1024/discriminator.h5.latest"
                        # transfer_GD_path = "/home/ec2-user/SageMaker/efs/model/gan-cifar10-012345678__v019__data_set=cifar10__class_list=012345678__architecture=brownlee__optimizer=RMSprop__learningrate=0.0001__batchsize=1024/generator.h5.latest"
                        D_num_frozen_layers = 0
                        GD_num_frozen_layers = 0
                        ## run
#                         try:
#                             run_train(x_train, object_shape, \
#                                       arch_choice, optimizer_type, learning_rate, \
#                                       wandb_config, logger, 
#                                       transfer_D_path, transfer_GD_path,
#                                       D_num_frozen_layers, GD_num_frozen_layers, True)
#                         except Exception as e:
#                             print(e)

    
        
    
    

project_name=gan-cifar10-012345678-allfreeze__v023	project_tag=__data_set=cifar10__class_list=012345678__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024
CREATION_FAILED=/home/ec2-user/SageMaker/efs/creation/gan-cifar10-012345678-allfreeze__v023__data_set=cifar10__class_list=012345678__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
CREATION_FAILED=/home/ec2-user/SageMaker/efs/model/gan-cifar10-012345678-allfreeze__v023__data_set=cifar10__class_list=012345678__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
x_train_before_class_filter= (50000, 32, 32, 3)
x_train_after_downsample= (4500, 32, 32, 3)
x_train_after_augmentation= (4500, 32, 32, 3)
x_train_after_class_filter= (4500, 32, 32, 3)


In [20]:
"""
MAJORITY CLASS PART2: Enhance CIFAR10 Generator for all majority classes, performing Data Augmentation 
"""

version = "25"
name = "gan-cifar10-class9-fromscratch-augmented10x"
project_name = (name + "__v%s") % str(format(int(version), '03d'))
global_config = SystemConfig(project_name)
wandb_config = global_config.get_config()
logger = logging.getLogger()    
  
# define hyper param space
hyperparam_space = {}  
hyperparam_space['data_set'] = ['cifar10'] #  'mnist'    
hyperparam_space['class_list'] = [9] #  0,1,2,3,4,5,6,7,8
hyperparam_space['architecture_choice'] = ['brownlee'] #  'atienza', 'atiber', 'brownlee_truck'
hyperparam_space['optimizer_type'] = ['Adam']  # ', 'RMSprop'
hyperparam_space['learning_rate'] = [0.0002]  # 0.01*10**(-2), 0.1*10**(-2), 0.001*10**(-2)
hyperparam_space['batch_size'] = [1024] # 128, 4096, 8192, 1024, 256, 


# iterate over hyper param space    
for dataset_name in hyperparam_space['data_set']:
    for arch_choice in hyperparam_space['architecture_choice']:
        wandb_config['architecture_choice'] = arch_choice
        for optimizer_type in hyperparam_space['optimizer_type']: 
            wandb_config['optimizer_type'] = optimizer_type
            for learning_rate in hyperparam_space['learning_rate']:
                wandb_config['learning_rate'] = learning_rate
                for train_batch_size in hyperparam_space['batch_size']:
                    wandb_config['train_batch_size'] = train_batch_size
                    ## path   
                    project_tag = '__data_set=' + dataset_name
                    class_string = ""
                    for k in sorted(hyperparam_space['class_list']):
                        class_string += str(k) 
                    project_tag += '__class_list=' + class_string
                    project_tag += '__architecture=' + arch_choice 
                    project_tag += '__optimizer=' + optimizer_type 
                    project_tag += '__learningrate='+ str(learning_rate) 
                    project_tag += '__batchsize=' + str(train_batch_size) 
                    wandb_config['project_tag'] = project_tag
                    print("project_name=" + project_name + "\t"+ "project_tag=" + project_tag)
                    ## file
                    global_config.set_log_file(logger, project_tag)
                    global_config.set_creation_folder(project_tag)
                    global_config.set_model_folder(project_tag)
                    ## data set
                    x_train, object_shape = get_data_set(dataset_name, hyperparam_space['class_list'],
                                                        is_augment = True, downsample_percent=0.1) 
                    ## transfer learning
                    transfer_D_path = ""
                    transfer_GD_path = ""
                    # transfer_D_path = "/home/ec2-user/SageMaker/Rosenblatt-AI/CS230/latest/discriminator.h5.1575816109.incomplete.9"
                    # transfer_GD_path = "/home/ec2-user/SageMaker/Rosenblatt-AI/CS230/latest/generator.h5.1575816096.incomplete.9"
                    D_num_frozen_layers = 0
                    GD_num_frozen_layers = 0 # 6
                    ## run
#                     try:
#                         run_train(x_train, object_shape, \
#                                   arch_choice, optimizer_type, learning_rate, \
#                                   wandb_config, logger, 
#                                   transfer_D_path, transfer_GD_path,
#                                   D_num_frozen_layers, GD_num_frozen_layers, True)
#                     except Exception as e:
#                         print(e)


project_name=gan-cifar10-class9-fromscratch-augmented10x__v025	project_tag=__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024
CREATION_FAILED=/home/ec2-user/SageMaker/efs/creation/gan-cifar10-class9-fromscratch-augmented10x__v025__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
CREATION_FAILED=/home/ec2-user/SageMaker/efs/model/gan-cifar10-class9-fromscratch-augmented10x__v025__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
x_train_before_class_filter= (50000, 32, 32, 3)
x_train_after_downsample= (500, 32, 32, 3)
x_train_after_augmentation= (5500, 32, 32, 3)
x_train_after_class_filter= (5500, 32, 32, 3)


In [ ]:
"""
Keras Nested Models Bug
"""
# https://github.com/keras-team/keras/pull/11847
# https://github.com/tensorflow/tensorflow/issues/27769

In [30]:
"""
EVALUATE GENERATOR: 
"""

version = "24"
name = "gan-cifar10-class9-trucky"
project_name = (name + "__v%s") % str(format(int(version), '03d'))
global_config = SystemConfig(project_name)
wandb_config = global_config.get_config()
logger = logging.getLogger()    
  
# define hyper param space
hyperparam_space = {}  
hyperparam_space['data_set'] = ['cifar10'] #  'mnist'    
hyperparam_space['class_list'] = [9] #  0,1,2,3,4,5,6,7,8
hyperparam_space['architecture_choice'] = ['brownlee'] #  'atienza', 'atiber', 'brownlee_truck'
hyperparam_space['optimizer_type'] = ['Adam']  # ', 'RMSprop'
hyperparam_space['learning_rate'] = [0.0002]  # 0.01*10**(-2), 0.1*10**(-2), 0.001*10**(-2)
hyperparam_space['batch_size'] = [1024] # 128, 4096, 8192, 1024, 256, 


# iterate over hyper param space    
for dataset_name in hyperparam_space['data_set']:
    for arch_choice in hyperparam_space['architecture_choice']:
        wandb_config['architecture_choice'] = arch_choice
        for optimizer_type in hyperparam_space['optimizer_type']: 
            wandb_config['optimizer_type'] = optimizer_type
            for learning_rate in hyperparam_space['learning_rate']:
                wandb_config['learning_rate'] = learning_rate
                for train_batch_size in hyperparam_space['batch_size']:
                    wandb_config['train_batch_size'] = train_batch_size
                    ## path   
                    project_tag = '__data_set=' + dataset_name
                    class_string = ""
                    for k in sorted(hyperparam_space['class_list']):
                        class_string += str(k) 
                    project_tag += '__class_list=' + class_string
                    project_tag += '__architecture=' + arch_choice 
                    project_tag += '__optimizer=' + optimizer_type 
                    project_tag += '__learningrate='+ str(learning_rate) 
                    project_tag += '__batchsize=' + str(train_batch_size) 
                    wandb_config['project_tag'] = project_tag
                    print("project_name=" + project_name + "\t"+ "project_tag=" + project_tag)
                    ## file
                    global_config.set_log_file(logger, project_tag)
                    global_config.set_creation_folder(project_tag)
                    global_config.set_model_folder(project_tag)
                    ## data set
                    x_train, object_shape = get_data_set(dataset_name, hyperparam_space['class_list'],
                                                        is_augment = True, downsample_percent=0.1) 
                    ## transfer learning
                    transfer_D_path = ""
                    transfer_GD_path = ""
#                     transfer_D_path = "/home/ec2-user/SageMaker/efs/model/gan-cifar10-class9-trucky__v024__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024/discriminator.h5.latest"
#                     transfer_GD_path = "/home/ec2-user/SageMaker/efs/model/gan-cifar10-class9-trucky__v024__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024/generator.h5.latest"                    
#                     transfer_D_path = "/home/ec2-user/SageMaker/efs/model/gan-cifar10-class9-trucky__v024__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024/discriminator.h5.1575766537"
#                     transfer_GD_path = "/home/ec2-user/SageMaker/efs/model/gan-cifar10-class9-trucky__v024__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024/generator.h5.1575766534"
                    D_num_frozen_layers = 0
                    GD_num_frozen_layers = 20
                    ## run
                    try:
                        g_a_n_trucky = \
                            run_train(x_train, object_shape, \
                                      arch_choice, optimizer_type, learning_rate, \
                                      wandb_config, logger, 
                                      transfer_D_path, transfer_GD_path,
                                      D_num_frozen_layers, GD_num_frozen_layers, False)
                        
                    except Exception as e:
                        print(e)


project_name=gan-cifar10-class9-trucky__v024	project_tag=__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024
CREATION_FAILED=/home/ec2-user/SageMaker/efs/creation/gan-cifar10-class9-trucky__v024__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
CREATION_FAILED=/home/ec2-user/SageMaker/efs/model/gan-cifar10-class9-trucky__v024__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
x_train_before_class_filter= (50000, 32, 32, 3)
x_train_after_downsample= (500, 32, 32, 3)
x_train_after_augmentation= (5500, 32, 32, 3)
x_train_after_class_filter= (5500, 32, 32, 3)





 NETWORK_NAME= D_
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu_69 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_70 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_71 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 4, 4, 256)         

In [31]:
"""
EVALUATE GENERATOR: 
"""

version = "25"
name = "gan-cifar10-class9-fromscratch-augmented10x"
project_name = (name + "__v%s") % str(format(int(version), '03d'))
global_config = SystemConfig(project_name)
wandb_config = global_config.get_config()
logger = logging.getLogger()    
  
# define hyper param space
hyperparam_space = {}  
hyperparam_space['data_set'] = ['cifar10'] #  'mnist'    
hyperparam_space['class_list'] = [9] #  0,1,2,3,4,5,6,7,8
hyperparam_space['architecture_choice'] = ['brownlee'] #  'atienza', 'atiber', 'brownlee_truck'
hyperparam_space['optimizer_type'] = ['Adam']  # ', 'RMSprop'
hyperparam_space['learning_rate'] = [0.0002]  # 0.01*10**(-2), 0.1*10**(-2), 0.001*10**(-2)
hyperparam_space['batch_size'] = [1024] # 128, 4096, 8192, 1024, 256, 


# iterate over hyper param space    
for dataset_name in hyperparam_space['data_set']:
    for arch_choice in hyperparam_space['architecture_choice']:
        wandb_config['architecture_choice'] = arch_choice
        for optimizer_type in hyperparam_space['optimizer_type']: 
            wandb_config['optimizer_type'] = optimizer_type
            for learning_rate in hyperparam_space['learning_rate']:
                wandb_config['learning_rate'] = learning_rate
                for train_batch_size in hyperparam_space['batch_size']:
                    wandb_config['train_batch_size'] = train_batch_size
                    ## path   
                    project_tag = '__data_set=' + dataset_name
                    class_string = ""
                    for k in sorted(hyperparam_space['class_list']):
                        class_string += str(k) 
                    project_tag += '__class_list=' + class_string
                    project_tag += '__architecture=' + arch_choice 
                    project_tag += '__optimizer=' + optimizer_type 
                    project_tag += '__learningrate='+ str(learning_rate) 
                    project_tag += '__batchsize=' + str(train_batch_size) 
                    wandb_config['project_tag'] = project_tag
                    print("project_name=" + project_name + "\t"+ "project_tag=" + project_tag)
                    ## file
                    global_config.set_log_file(logger, project_tag)
                    global_config.set_creation_folder(project_tag)
                    global_config.set_model_folder(project_tag)
                    ## data set
                    x_train, object_shape = get_data_set(dataset_name, hyperparam_space['class_list'],
                                                        is_augment = True, downsample_percent=0.1) 
                    ## transfer learning
                    transfer_D_path = ""
                    transfer_GD_path = ""
                    D_num_frozen_layers = 0
                    GD_num_frozen_layers = 0 # 6
                    ## run
                    try:
                        g_a_n_10x = \
                            run_train(x_train, object_shape, \
                                      arch_choice, optimizer_type, learning_rate, \
                                      wandb_config, logger, 
                                      transfer_D_path, transfer_GD_path,
                                      D_num_frozen_layers, GD_num_frozen_layers, False)
                        
                    except Exception as e:
                        print(e)


project_name=gan-cifar10-class9-fromscratch-augmented10x__v025	project_tag=__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024
CREATION_FAILED=/home/ec2-user/SageMaker/efs/creation/gan-cifar10-class9-fromscratch-augmented10x__v025__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
CREATION_FAILED=/home/ec2-user/SageMaker/efs/model/gan-cifar10-class9-fromscratch-augmented10x__v025__data_set=cifar10__class_list=9__architecture=brownlee__optimizer=Adam__learningrate=0.0002__batchsize=1024 failed
x_train_before_class_filter= (50000, 32, 32, 3)
x_train_after_downsample= (500, 32, 32, 3)
x_train_after_augmentation= (5500, 32, 32, 3)
x_train_after_class_filter= (5500, 32, 32, 3)





 NETWORK_NAME= D_
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu_77 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
leaky_re_lu_78 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
leaky_re_lu_79 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 4, 4, 256)         

In [1]:
"""
10x Augmented Generation
"""
num_samples = 4500 * 50
G_input_n = 100
z_noise = np.random.uniform(-1.0, 1.0, size=[num_samples, G_input_n])
G_of_z_fake_objects = g_a_n_10x.get_G_model().predict(z_noise)

denormalized_fake = (G_of_z_fake_objects + 1.0) / 2.0

to_plot = denormalized_fake[0]

outil.plot_one_object(to_plot, "save_me_fake.png", "./")

print("SIZE_OF_FAKE=", G_of_z_fake_objects.shape, to_plot.shape)
# print(G_of_z_fake_objects)

np.save('/home/ec2-user/SageMaker/efs/generated_examples_class_9.png', denormalized_fake)


NameError: name 'np' is not defined

In [ ]:
# np.array(denormalized_fake)